In [1]:
%reset -f

In [2]:
%reload_ext sql
%sql sqlite:///../ontology/tmp/merged.db

In [3]:
%sql SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///../ontology/tmp/merged.db
Done.


name
term_association
has_oio_synonym_statement
anonymous_expression
anonymous_class_expression
anonymous_property_expression
anonymous_individual_expression
owl_restriction
owl_complex_axiom
prefix
statements


In [4]:
import pandas as pd
from os.path import join
import textdistance

def save_csv(query, fn):
    dir_name = "dataframes"
    df = pd.DataFrame(query)
    df.to_csv(join(dir_name,fn), sep='\t', index = False)
    return df

def add_distance(df, col_name, txt_dist_pkg):
    df\
    .insert(\
            len(df.columns),\
            col_name,\
            df.apply\
            (\
                lambda x: txt_dist_pkg\
                 (\
                    x.subject_value.lower(), x.object_value.lower()\
                 ),\
                axis=1,\
            )\
           )

In [5]:
query = %sql SELECT * FROM statements WHERE subject LIKE "%MONDO:%" AND object LIKE "%ICD10%"
mondo_2_icd_df = save_csv(query, "mondo_2_icd.tsv")
mondo_2_icd_df.head()


 * sqlite:///../ontology/tmp/merged.db
Done.


,stanza,subject,predicate,object,value,datatype,language
0,MONDO:0000087,MONDO:0000087,skos:narrowMatch,<https://icd.codes/icd10cm/Q04.3>,None,None,None
1,MONDO:0000088,MONDO:0000088,skos:exactMatch,<https://icd.codes/icd10cm/E30.1>,None,None,None
2,MONDO:0000088,MONDO:0000088,skos:exactMatch,<http://purl.bioontology.org/ontology/ICD10CM/...,None,None,None
3,MONDO:0000107,MONDO:0000107,skos:narrowMatch,<https://icd.codes/icd10cm/Q75.8>,None,None,None
4,MONDO:0000110,MONDO:0000110,skos:narrowMatch,<https://icd.codes/icd10cm/Q30.2>,None,None,None


In [6]:
query = %sql SELECT * FROM statements WHERE subject LIKE "%MONDO:%" OR subject LIKE "%ICD10%" AND object LIKE "%MONDO:%" OR object LIKE "%ICD10%"
mondo_icd_2_mondo_icd_df = save_csv(query, "mondo_icd_2_mondo_icd.tsv")
mondo_icd_2_mondo_icd_df.head()


 * sqlite:///../ontology/tmp/merged.db
Done.


,stanza,subject,predicate,object,value,datatype,language
0,MONDO:0100332,MONDO:0100332,rdfs:label,None,disease has primary infectious agent,xsd:string,None
1,MONDO:0100332,MONDO:0100332,oio:id,None,MONDO:0100332,xsd:string,None
2,MONDO:0100332,MONDO:0100332,oio:created_by,None,https://orcid.org/0000-0001-5208-3432,xsd:string,None
3,MONDO:0100332,MONDO:0100332,rdfs:subPropertyOf,RO:0014001,None,None,None
4,MONDO:0100332,MONDO:0100332,rdf:type,owl:ObjectProperty,None,None,None


In [7]:
query = %sql SELECT * FROM statements WHERE (subject LIKE "%MONDO:%" OR subject LIKE "%icd10cm%") AND predicate IN ("rdfs:label", "oio:hasExactSynonym")
mondo_icd_label_exact_synonym_df = save_csv(query, "mondo_icd_label_exact_synonym.tsv")
mondo_icd_label_exact_synonym_df.head()


 * sqlite:///../ontology/tmp/merged.db
Done.


,stanza,subject,predicate,object,value,datatype,language
0,<http://purl.bioontology.org/ontology/ICD10CM/...,<http://purl.bioontology.org/ontology/ICD10CM/...,oio:hasExactSynonym,None,diseases generally recognized as communicable ...,None,en
1,<http://purl.bioontology.org/ontology/ICD10CM/...,<http://purl.bioontology.org/ontology/ICD10CM/...,oio:hasExactSynonym,None,Classical cholera,None,en
2,<http://purl.bioontology.org/ontology/ICD10CM/...,<http://purl.bioontology.org/ontology/ICD10CM/...,oio:hasExactSynonym,None,Cholera eltor,None,en
3,<http://purl.bioontology.org/ontology/ICD10CM/...,<http://purl.bioontology.org/ontology/ICD10CM/...,oio:hasExactSynonym,None,Infection due to Salmonella typhi,None,en
4,<http://purl.bioontology.org/ontology/ICD10CM/...,<http://purl.bioontology.org/ontology/ICD10CM/...,oio:hasExactSynonym,None,Typhoid myocarditis,None,en


In [8]:
%sql SELECT * FROM statements WHERE (subject LIKE "%icd10cm%") AND predicate IN ("rdfs:label", "oio:hasExactSynonym") LIMIT 10


 * sqlite:///../ontology/tmp/merged.db
Done.


stanza,subject,predicate,object,value,datatype,language
<http://purl.bioontology.org/ontology/ICD10CM/A00-B99>,<http://purl.bioontology.org/ontology/ICD10CM/A00-B99>,oio:hasExactSynonym,None,diseases generally recognized as communicable or transmissible,None,en
<http://purl.bioontology.org/ontology/ICD10CM/A00.0>,<http://purl.bioontology.org/ontology/ICD10CM/A00.0>,oio:hasExactSynonym,None,Classical cholera,None,en
<http://purl.bioontology.org/ontology/ICD10CM/A00.1>,<http://purl.bioontology.org/ontology/ICD10CM/A00.1>,oio:hasExactSynonym,None,Cholera eltor,None,en
<http://purl.bioontology.org/ontology/ICD10CM/A01.0>,<http://purl.bioontology.org/ontology/ICD10CM/A01.0>,oio:hasExactSynonym,None,Infection due to Salmonella typhi,None,en
<http://purl.bioontology.org/ontology/ICD10CM/A01.02>,<http://purl.bioontology.org/ontology/ICD10CM/A01.02>,oio:hasExactSynonym,None,Typhoid myocarditis,None,en
<http://purl.bioontology.org/ontology/ICD10CM/A01.02>,<http://purl.bioontology.org/ontology/ICD10CM/A01.02>,oio:hasExactSynonym,None,Typhoid endocarditis,None,en
<http://purl.bioontology.org/ontology/ICD10CM/A01.4>,<http://purl.bioontology.org/ontology/ICD10CM/A01.4>,oio:hasExactSynonym,None,Infection due to Salmonella paratyphi NOS,None,en
<http://purl.bioontology.org/ontology/ICD10CM/A02>,<http://purl.bioontology.org/ontology/ICD10CM/A02>,oio:hasExactSynonym,None,infection or foodborne intoxication due to any Salmonella species other than S. typhi and S. paratyphi,None,en
<http://purl.bioontology.org/ontology/ICD10CM/A02.0>,<http://purl.bioontology.org/ontology/ICD10CM/A02.0>,oio:hasExactSynonym,None,Salmonellosis,None,en
<http://purl.bioontology.org/ontology/ICD10CM/A02.25>,<http://purl.bioontology.org/ontology/ICD10CM/A02.25>,oio:hasExactSynonym,None,Salmonella tubulo-interstitial nephropathy,None,en


In [9]:
%sql SELECT * FROM entailed_edge WHERE object LIKE "%ICD10%" LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


subject,predicate,object
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M60-M63>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M61.05>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M61.0>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M00-M99>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M61.052>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M61>
<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1A>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/T36.3>
<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1A>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1>
<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1A>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/T36-T50>
<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1A>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/S00-T88>


In [10]:
%sql SELECT * FROM entailed_edge WHERE subject LIKE "%MONDO_%" AND object LIKE "%ICD10%" LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


subject,predicate,object


In [11]:
%sql SELECT * FROM entailed_edge WHERE subject LIKE "%ICD10%" AND object LIKE "%MONDO_%"  LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


subject,predicate,object


In [12]:
%sql SELECT * FROM entailed_edge WHERE subject LIKE "%ICD10%" LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


subject,predicate,object
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M60-M63>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M61.05>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M61.0>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M00-M99>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M61.052>
<http://purl.bioontology.org/ontology/ICD10CM/M61.052>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/M61>
<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1A>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/T36.3>
<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1A>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1>
<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1A>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/T36-T50>
<http://purl.bioontology.org/ontology/ICD10CM/T36.3X1A>,rdfs:subClassOf,<http://purl.bioontology.org/ontology/ICD10CM/S00-T88>


In [13]:
%sql SELECT * FROM rdfs_label_statement LIMIT 10

 * sqlite:///../ontology/tmp/merged.db
Done.


stanza,subject,predicate,object,value,datatype,language
IAO:0000111,IAO:0000111,rdfs:label,None,editor preferred term,None,en
IAO:0000112,IAO:0000112,rdfs:label,None,example of usage,None,en
IAO:0000113,IAO:0000113,rdfs:label,None,in branch,None,en
IAO:0000114,IAO:0000114,rdfs:label,None,has curation status,None,en
IAO:0000115,IAO:0000115,rdfs:label,None,definition,xsd:string,None
IAO:0000115,IAO:0000115,rdfs:label,None,definition,None,en
IAO:0000116,IAO:0000116,rdfs:label,None,editor note,None,en
IAO:0000117,IAO:0000117,rdfs:label,None,term editor,None,en
IAO:0000118,IAO:0000118,rdfs:label,None,alternative term,None,en
IAO:0000118,IAO:0000118,rdfs:label,None,alternative label,None,en


## Post-process some DataFrames

In [14]:
full_table_query = %sql SELECT subject, predicate, object, value, datatype FROM rdfs_label_statement
full_table_df = pd.DataFrame(full_table_query)
full_table_df.head()

 * sqlite:///../ontology/tmp/merged.db
Done.


,subject,predicate,object,value,datatype
0,IAO:0000111,rdfs:label,None,editor preferred term,None
1,IAO:0000112,rdfs:label,None,example of usage,None
2,IAO:0000113,rdfs:label,None,in branch,None
3,IAO:0000114,rdfs:label,None,has curation status,None
4,IAO:0000115,rdfs:label,None,definition,xsd:string


In [15]:
full_table_df_with_values = full_table_df[full_table_df['value'].values != None]
subject_value_df = full_table_df_with_values[['subject', "value"]]
subject_value_df.head()

,subject,value
0,IAO:0000111,editor preferred term
1,IAO:0000112,example of usage
2,IAO:0000113,in branch
3,IAO:0000114,has curation status
4,IAO:0000115,definition


In [16]:
cols_to_drop = ['stanza','value', 'datatype','language']
mondo_2_icd_df_subset = mondo_2_icd_df.drop(columns=cols_to_drop)
mondo_2_icd_df_subset.head()

,subject,predicate,object
0,MONDO:0000087,skos:narrowMatch,<https://icd.codes/icd10cm/Q04.3>
1,MONDO:0000088,skos:exactMatch,<https://icd.codes/icd10cm/E30.1>
2,MONDO:0000088,skos:exactMatch,<http://purl.bioontology.org/ontology/ICD10CM/...
3,MONDO:0000107,skos:narrowMatch,<https://icd.codes/icd10cm/Q75.8>
4,MONDO:0000110,skos:narrowMatch,<https://icd.codes/icd10cm/Q30.2>


In [17]:
mondo_2_icd_subset_with_values = mondo_2_icd_df_subset\
                                .merge(subject_value_df, how="left", on="subject")\
                                .rename(columns={'value': "subject_value"})
mondo_2_icd_subset_with_values.head()


,subject,predicate,object,subject_value
0,MONDO:0000087,skos:narrowMatch,<https://icd.codes/icd10cm/Q04.3>,polymicrogyria
1,MONDO:0000088,skos:exactMatch,<https://icd.codes/icd10cm/E30.1>,precocious puberty
2,MONDO:0000088,skos:exactMatch,<http://purl.bioontology.org/ontology/ICD10CM/...,precocious puberty
3,MONDO:0000107,skos:narrowMatch,<https://icd.codes/icd10cm/Q75.8>,auriculocondylar syndrome
4,MONDO:0000110,skos:narrowMatch,<https://icd.codes/icd10cm/Q30.2>,bifid nose


In [18]:
mondo_2_icd_values_df = mondo_2_icd_subset_with_values\
                        .merge(subject_value_df, how="left", left_on="object", right_on="subject")\
                        .rename(columns={'value': "object_value", "subject_x": "subject"})\
                        .drop(columns=['subject_y'])\
                        .fillna("")
        

mondo_2_icd_values_df.head()


,subject,predicate,object,subject_value,object_value
0,MONDO:0000087,skos:narrowMatch,<https://icd.codes/icd10cm/Q04.3>,polymicrogyria,
1,MONDO:0000088,skos:exactMatch,<https://icd.codes/icd10cm/E30.1>,precocious puberty,
2,MONDO:0000088,skos:exactMatch,<http://purl.bioontology.org/ontology/ICD10CM/...,precocious puberty,Precocious puberty
3,MONDO:0000107,skos:narrowMatch,<https://icd.codes/icd10cm/Q75.8>,auriculocondylar syndrome,
4,MONDO:0000110,skos:narrowMatch,<https://icd.codes/icd10cm/Q30.2>,bifid nose,


In [19]:
mondo_2_icd_values_df = mondo_2_icd_values_df[['subject', 'subject_value', 'predicate', 'object', 'object_value' ]]
mondo_2_icd_values_df.to_csv(join("dataframes","mondo2icd_with_values.tsv"), sep='\t', index=False)


## Subject and Object value textdistances

In [20]:
# Add Levenshtein distance [ 0: Perfect match]
add_distance(mondo_2_icd_values_df, "levenshtein_dist", textdistance.levenshtein.distance)

In [21]:
# Add Jaccard Index [ 0: Perfect match]
add_distance(mondo_2_icd_values_df, "jaccard_index", textdistance.jaccard.distance)

In [22]:
# Add Monge-Elkan Distance [ 0: Perfect match]
add_distance(mondo_2_icd_values_df, "monge_elkan", textdistance.monge_elkan.distance)
mondo_2_icd_values_df.to_csv(join("dataframes","mondo2icd_with_distances.tsv"), sep='\t', index=False)
mondo_2_icd_values_df.head()

,subject,subject_value,predicate,object,object_value,levenshtein_dist,jaccard_index,monge_elkan
0,MONDO:0000087,polymicrogyria,skos:narrowMatch,<https://icd.codes/icd10cm/Q04.3>,,14,1.0,2.0
1,MONDO:0000088,precocious puberty,skos:exactMatch,<https://icd.codes/icd10cm/E30.1>,,18,1.0,2.0
2,MONDO:0000088,precocious puberty,skos:exactMatch,<http://purl.bioontology.org/ontology/ICD10CM/...,Precocious puberty,0,0.0,0.0
3,MONDO:0000107,auriculocondylar syndrome,skos:narrowMatch,<https://icd.codes/icd10cm/Q75.8>,,25,1.0,2.0
4,MONDO:0000110,bifid nose,skos:narrowMatch,<https://icd.codes/icd10cm/Q30.2>,,10,1.0,2.0
